## A disciplined approach to cross-validation in a Kaggle (Inclass) competition

In [32]:
# Import libraries and set desired options
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import hstack
# !pip install eli5
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display_html

In [5]:
PATH_TO_DATA = '../../data/alice/'
SEED = 17

In [6]:
path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv')
path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv')
path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl')

In [7]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

### Loading data

In [8]:
times = ['time%s' % i for i in range(1, 11)]
train_df = pd.read_csv(path_to_train,
                   index_col='session_id', parse_dates=times)
test_df = pd.read_csv(path_to_test,
                  index_col='session_id', parse_dates=times)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# read site -> id mapping provided by competition organizers 
with open(path_to_site_dict, 'rb') as f:
    site2id = pickle.load(f)
# create an inverse id _> site mapping
id2site = {v:k for (k, v) in site2id.items()}
# we treat site with id 0 as "unknown"
id2site[0] = 'unknown'

# Transform data into format which can be fed into TfidfVectorizer
# This time we prefer to represent sessions with site names, not site ids. 
# It's less efficient but thus it'll be more convenient to interpret model weights.
sites = ['site%s' % i for i in range(1, 11)]
train_sessions = train_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                 ' '.join([id2site[i] for i in row]), axis=1).tolist()
test_sessions = test_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                 ' '.join([id2site[i] for i in row]), axis=1).tolist()

In [9]:
# we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
# so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
# to be split into 'mail', 'google' and 'com')
vectorizer_params={'ngram_range': (1, 5), 
                   'max_features': 50000,
                   'tokenizer': lambda s: s.split()}
vectorizer = TfidfVectorizer(**vectorizer_params)

X_train_sites = vectorizer.fit_transform(train_sessions)
X_test_sites = vectorizer.transform(test_sessions)
y_train = train_df['target'].astype('int').values

# we'll need site visit times for further feature engineering
train_times, test_times = train_df[times], test_df[times]

In [10]:
vectorizer.get_feature_names()[:10]

['0.academia-assets.com',
 '0.docs.google.com',
 '0.docs.google.com 0.docs.google.com',
 '0.docs.google.com 0.docs.google.com 0.docs.google.com',
 '0.docs.google.com 0.docs.google.com 0.docs.google.com 0.docs.google.com',
 '0.docs.google.com 0.docs.google.com 0.drive.google.com',
 '0.docs.google.com 0.docs.google.com apis.google.com',
 '0.docs.google.com 0.docs.google.com docs.google.com',
 '0.docs.google.com 0.drive.google.com',
 '0.docs.google.com 0.drive.google.com 0.docs.google.com']

## Extracting features

In [11]:
# check the sparse matrix shape
X_train_sites.shape, X_test_sites.shape

((253561, 50000), (82797, 50000))

In [12]:
# check the time DataFrames shape
train_times.shape, test_times.shape, y_train.shape

((253561, 10), (82797, 10), (253561,))

### pipeline for dummy variables
time_feat = pd.DataFrame()
time_feat['hour'] = train_times['time1'].apply(lambda ts: ts.hour)
hour = time_feat['hour']
time_feat['morning'] = ((hour >= 7) & (hour <= 11)).astype(np.int8)
time_feat['day'] = ((hour >= 12) & (hour <= 18)).astype(np.int8)
time_feat['evening'] = ((hour >= 19) & (hour <= 23)).astype(np.int8)
time_feat['night'] = ((hour >= 0) & (hour <=6)).astype(np.int8)
time_feat['month'] = train_times['time1'].apply(lambda ts: ts.month).astype(np.int8)
time_feat['year'] = train_times['time1'].apply(lambda ts: ts.year).astype(np.int8)
time_feat['dayofweek'] = train_times['time1'].apply(lambda ts: ts.dayofweek).astype(np.int8)
time_feat['weekend'] = train_times['time1'].apply(lambda ts: ts.dayofweek > 5).astype(np.int8)
time_feat['yyyymm'] = train_times['time1'].apply(lambda ts: ts.year * 100 + ts.month).astype(np.int32)

tft = pd.DataFrame(train_times['time1'].apply(lambda ts: ts.hour).astype(np.int8))
hour_list = np.arange(0,24)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(hour_list.reshape(-1,1))
dummy_hour = enc.transform(tft.values)
feat = hstack([time_feat, dummy_hour])
### end of dummy pipeline

def add_time_features(times, X_sparse, df):
    # LB = 0.957 (C=3.0),  CV = 0.92  +-  0.05
    
    hour = times['time1'].apply(lambda ts: ts.hour).astype(np.int8)
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    
    day_of_week = times['time1'].apply(lambda t: t.weekday()).values.reshape(-1, 1)
    weekend = times['time1'].apply(lambda ts: ts.dayofweek > 5).astype(np.int8).values.reshape(-1,1)
      
    month = times['time1'].apply(lambda t: t.month).values.reshape(-1,1)
    yyyymm = times['time1'].apply(lambda ts: ts.year * 100 + ts.month).astype(np.float).values.reshape(-1, 1)
    yyyymm_sc = StandardScaler().fit_transform(yyyymm)
    
    duration = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]')\
                    .astype(np.float).values.reshape(-1, 1)
    duration_sc = StandardScaler().fit_transform(duration)
    
    hosts = pd.DataFrame(df['site1'].apply(lambda i: id2site[i]))
    hosts['split'] = hosts['site1'].str.split('.')
    domain = hosts['split'].map(lambda x: x[-1])
    typical_domain = domain.map(lambda x: x in ('com', 'fr', 'net', 'uk', 'org', 'tv'))\
                                                        .astype(np.int).values.reshape(-1,1)
    
    X = hstack([X_sparse, morning, day, evening, night,yyyymm_sc, 
                duration_sc, day_of_week, weekend, month, typical_domain])
    
    return X

In [70]:
def add_time_features(times, X_sparse, df):
    
    hour = times['time1'].apply(lambda ts: ts.hour).astype(np.int8)
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    
    weekend = times['time1'].apply(lambda ts: ts.dayofweek > 5).astype(np.int8).values.reshape(-1,1)

    day_of_week = times['time1'].apply(lambda t: t.weekday()).values.reshape(-1, 1)
    day_of_week_sc = StandardScaler().fit_transform(day_of_week)
    
    month = times['time1'].apply(lambda t: t.month).values.reshape(-1,1)
    month_sc = StandardScaler().fit_transform(month)
    
    yyyymm = times['time1'].apply(lambda ts: ts.year * 100 + ts.month).astype(np.float).values.reshape(-1, 1)
    yyyymm_sc = StandardScaler().fit_transform(yyyymm)
    
    duration = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]')\
                    .astype(np.float).values.reshape(-1, 1)
    duration_sc = PowerTransformer().fit_transform(duration)
    
    hosts = pd.DataFrame(df['site1'].apply(lambda i: id2site[i]))
    hosts['split'] = hosts['site1'].str.split('.')
    domain = hosts['split'].map(lambda x: x[-1])
    typical_domain = domain.map(lambda x: x in ('com', 'fr', 'net', 'uk', 'org', 'tv'))\
                                                        .astype(np.int).values.reshape(-1,1)
    
    X = hstack([X_sparse, morning, day, evening, night,yyyymm_sc, 
                duration_sc, day_of_week_sc, weekend, month_sc, typical_domain])
    
    feat_name = ['morning', 'day', 'evening', 'night', 'yyyymm_sc', 
                'duration_sc', 'day_of_week_sc', 'weekend', 'month_sc', 'typical_domain']
    return X, feat_name

In [71]:
X_train_with_times1, feat_names = add_time_features(train_times, X_train_sites, train_df)
X_test_with_times1, _ = add_time_features(test_times, X_test_sites, test_df)

/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: D

In [72]:
X_train_with_times1.shape, X_test_with_times1.shape

((253561, 50010), (82797, 50010))

### Time series cross-validation

In [43]:
time_split = TimeSeriesSplit(n_splits=10)

In [44]:
logit = LogisticRegression(C=1, random_state=SEED, solver='liblinear')
xgb_clf = XGBClassifier(random_state=SEED, n_jobs=-1)

In [73]:
def train_and_score(model, X_train, y_train, X_test, site_feature_names=vectorizer.get_feature_names(), 
                      new_feature_names=None, cv=time_split, scoring='roc_auc',
                      top_n_features_to_show=30):
    
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring=scoring, n_jobs=-1)
    print('CV scores', cv_scores)
    print('CV mean: {}, CV std: {}'.format(cv_scores.mean(), cv_scores.std()))
    model.fit(X_train, y_train)
    
    if new_feature_names:
        all_feature_names = site_feature_names + new_feature_names 
    else: 
        all_feature_names = site_feature_names
    
    display_html(eli5.show_weights(estimator=model, 
                  feature_names=all_feature_names, top=top_n_features_to_show))
    
    if new_feature_names:
        print('New feature weights:')
    
        print(pd.DataFrame({'feature': new_feature_names, 
                        'coef': model.coef_.flatten()[-len(new_feature_names):]}))
    
  
    return cv_scores

In [74]:
cv_scores1 = train_and_score(model=logit, X_train=X_train_with_times1, y_train=y_train, 
                      X_test=X_test_with_times1, new_feature_names=feat_names,
                        site_feature_names=vectorizer.get_feature_names(), cv=time_split)

CV scores [0.80341042 0.82298081 0.94834361 0.96143459 0.91443285 0.95682408
 0.92800749 0.94891589 0.95703609 0.96836846]
CV mean: 0.9209754277162295, CV std: 0.05613818519879441


Weight?,Feature
+5.139,youwatch.org
+4.994,www.express.co.uk
+4.980,cid-ed6c3e6a5c6608a4.users.storage.live.com
+4.961,vk.com
+4.791,www.info-jeunes.net
+4.424,www.melty.fr
+4.397,fr.glee.wikia.com
+4.357,www.audienceinsights.net
+4.050,www.banque-chalus.fr
+3.937,api.bing.com


New feature weights:
          feature      coef
0         morning -3.279178
1             day  0.440634
2         evening -2.816813
3           night  0.000000
4       yyyymm_sc -0.303011
5     duration_sc -0.184063
6  day_of_week_sc -0.578521
7         weekend -1.898722
8        month_sc  0.158926
9  typical_domain  0.081872


New feature weights:
           feature      coef
0          morning -3.483272
1              day  0.173576
2          evening -3.141868
3            night  0.000000
4        yyyymm_sc -0.084103
5      duration_sc -0.274467
6      day_of_week -0.309776
7        alice_day  2.050477
8          weekend -0.783444
9         month_sc  0.447099
10  typical_domain  0.057839

### Tuning parametres

In [75]:
# here we've already narrowed down c_values to such a range.
# typically, you would start with a wider range of values to check
c_values = np.logspace(-1, 1, 10)
param_grid = {'C': c_values}

In [76]:
logit_grid_searcher = GridSearchCV(estimator=logit, param_grid=param_grid,
                                  scoring='roc_auc', n_jobs=2, cv=time_split, verbose=1)

In [77]:
%%time
logit_grid_searcher.fit(X_train_with_times1, y_train); 

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/andrey/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/a

CPU times: user 13.8 s, sys: 370 ms, total: 14.2 s
Wall time: 2min 18s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'C': array([ 0.1    ,  0.16681,  0.27826,  0.46416,  0.77426,  1.29155,
        2.15443,  3.59381,  5.99484, 10.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [78]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9237907987572158, {'C': 3.593813663804626})

param_grid_xgb = {'max_depth': [4, 5, 6], 
                  'min_child_weight': [1, 2, 3, 4]}

xgb_grid_searcher = GridSearchCV(estimator=xgb_clf, param_grid=param_grid_xgb,
                                  scoring='roc_auc', n_jobs=2, cv=5, verbose=1)
%%time
xgb_grid_searcher.fit(X_train_with_times1, y_train); 

xgb_grid_searcher.best_score_, xgb_grid_searcher.best_params_
(0.8142830042695925, {'max_depth': 6, 'min_child_weight': 2})

In [79]:
final_model = logit_grid_searcher.best_estimator_

### Submit result

In [80]:
test_pred = final_model.predict_proba(X_test_with_times1)[:, 1]
write_to_submission_file(test_pred, 'subm9.csv') 